# Análise dos dados de periódicos da Hemerotéca Digital Brasileira da Biblioteca Nacional

Os dados foram disponibilizados para pesquisa pela equipe da BNDigital em formato xml. O arquivo foi exportado da base do sistema de periódicos digitalizados e se encontra no padrão MARCXML (MAchine-Readable Cataloging XML). O arquivo pode ser acessado [aqui](/repositorios/BND-BR/data/exp_per_marcxml.xml).

Agradeço a Vinicius Pontes Martins do Setor de Gestão de Programas e Inovação (SGPI) da BNDigital pela disponibilização dos dados.

## Analisando e Filtrando o xml

Aqui vamos realizar realizar o *parse* dos dados contidos no arquivo xml e filtrar os dados que serão utilizados para análise.

Utilizaremos a biblioteca `xml.etree.ElementTree` para realizar o *parse* do arquivo xml. A biblioteca é nativa do Python e não necessita de instalação.

In [1]:
# importar bibliotecas
import xml.etree.ElementTree as ET

In [2]:
# ler arquivo xml
tree = ET.parse('./data/exp_per_marcxml.xml')

Após realizarmos o *parse* do ficheiro, podemos contar quantos registros exixtem na àrvore de elementos:

In [3]:
# contar número de registros
root = tree.getroot()
print(len(root))

7685


### Subcampos utilizados

Após a leitura do arquivos, selecionei sete subcampos para análise (serão listadas a tag e o subcampo de acordo com a estrutura do xml):

- tag 245; subcampo a: Título do periódico
- tag 245; subcampo b: Subtítulo do periódico
- tag 260; subcampo a: Local de publicação
- tag 260; subcampo b: Editora
- tag 260; subcampo c: Período de publicação
- tag 310; subcampo a: Periodicidade da publicação
- tag 546; subcampo a: Idioma da publicação

A seleção desses elementos buscou possibilitar a comparação com os dados disponibilizados pela BND-PT, conforme descrito no [notebook](..///BND-PT/escopo.ipynb).


Vamos criar um ficheiro csv com os dados selecionados. Para isso, vamos utilizar a biblioteca `csv` do Python. 

Primeiro criamos uma função para encontrar os subcampos desejados:

In [7]:
# função para encontrar o valor de um campo
def find_value(record, tag,code):
    return record.find(f"./datafield[@tag='{tag}']/subfield[@code='{code}']")

Em seguida, criamos o ficheiro csv e escrevemos os dados:

In [8]:
# criar um csv com os dados selecionados
import csv

with open('./data/complete_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writerow(['title', 'subtitle', 'place', 'period', 'editor', 'periodicity', 'language'])
    for record in root:
        title = find_value(record, '245', 'a')
        subtitle = find_value(record, '245', 'b')
        place = find_value(record, '260', 'a')
        period = find_value(record, '260', 'c')
        editor = find_value(record, '260', 'b')
        periodicity = find_value(record, '310', 'a')
        language = find_value(record, '546', 'a')
        if title is not None:
            title = title.text
        if subtitle is not None:
            subtitle = subtitle.text
        if place is not None:
            place = place.text
        if period is not None:
            period = period.text
        if editor is not None:
            editor = editor.text
        if periodicity is not None:
            periodicity = periodicity.text
        if language is not None:
            language = language.text
        writer.writerow([title, subtitle, place, period, editor, periodicity, language])

## Apresentação dos dados

A partir do ficheiro csv gerado na célula anterior, vamos apresentar os dados da HDB buscando uma compreensão geral do seu acervo de periódicos digitalizados.

Os dados serão analisados com a biblioteca `pandas` e apresentados com a biblioteca `plotly`.

In [9]:
# importar bibliotecas
import pandas as pd
import plotly.express as px

## Dados gerais do acervo

Vamos criar um dataframe com os dados do csv e apresentar sua estrutura e informações gerais.

In [25]:
# importar dataset e criar dataframe
df = pd.read_csv('./data/complete_data.csv', encoding='utf-8')

Para termos uma ideia geral do dataframe, vamos ver as primeiras 10 linhas do *dataframe*:

In [26]:
df.head()

,title,subtitle,place,period,editor,periodicity,language
0,O Abolicionista Paraense,NaN,"Belém, PA",1883-,Typ. da Provincia do Para,Semanal,por
1,O Abolicionista,propriedade de uma associação,"São Luis, MA",1885,Typ. do Abolicionista,NaN,por
2,O Academico,"periodico scientifico, litterario e especialme...","Rio de Janeiro, RJ",1855-,"Typ. Fluminense, de D.L. dos Santos",NaN,por
3,A Actualidade,orgao do Partido Liberal,"Ouro Preto, MG",1878-[1882],Typ. de Jose Egydio da Silca Campos,3 vezes por semana,NaN
4,A Actualidade,"periodico imparcial, litterario, critico e not...",Maranhão,1900-,Typ. de Antonio Pereira Ramos d'Almeida e C. S...,3 vezes por mês,por


O *dataframe* é composto pelas seguintes colunas:

In [27]:
# mostrar colunas em lista
df.columns.tolist()

['title', 'subtitle', 'place', 'period', 'editor', 'periodicity', 'language']

In [28]:
# contar número de registros
df.count()

title          7685
subtitle       3666
place          7608
period         7265
editor         5675
periodicity    5901
language       6614
dtype: int64

Percebemos que o dataframe conta com 7685 periódicos. Os dados das demais colunas variam, e as colunas período e local de publicação são as mais completas.

### Idiomas

Vamos contar os idiomas dos periódicos digitalizados, mas primeiro vamos limpar os dados da coluna 'language'.

In [29]:
# limpar dados de idioma
# lista de termos para substituir
replace_por = ['Texto em português', 'Texto em portugues', 'Português', 'Em português', 'Texto em português e alguns textos em francês']
replace_spa = ['Texto em espanhol', 'esp']
replace_fre = ['Texto em frances']

# substituir termos
df['language'] = df['language'].replace(replace_por, 'por')
df['language'] = df['language'].replace(replace_spa, 'spa')
df['language'] = df['language'].replace(replace_fre, 'fre')

Vamos avaliar quantos registros não possuem idioma definido:

In [30]:
# contar quantos registros não possuem idioma
df['language'].isnull().sum()

1071

In [31]:
# calcular a porcentagem de registros sem idioma
df['language'].isnull().sum() / len(df) * 100

13.936239427456082

Dos 7685 registros, 1071 não possuem idioma definido no dataframe, o que corresponde a 13,93% do total.

Vamos incluir o valor 'Não definido' para os registros sem idioma definido:

In [32]:
# incluir valor 'não definido' para registros sem idioma
df['language'] = df['language'].fillna('não definido')

Agora uma contagem dos idiomas dos periódicos será mais precisa:

In [33]:
# criar dataframe com a contagem de idiomas
df_lang = df['language'].value_counts().rename_axis('Idioma').reset_index(name='quantidade')
df_lang

,Idioma,quantidade
0,por,6462
1,não definido,1071
2,ger,61
3,ita,33
4,fre,27
5,spa,17
6,eng,10
7,ara,2
8,syr,1
9,yid,1


Em termos de porcentagem, podemos ver que o idioma predominante é o português, com 84% dos periódicos. Em seguida, temos os periódicos sem idioma definido, 13,9%, seguidos de alemão, italiano, francês, espanhol e inglês, cada um com menos de 1% cada.

In [34]:
# porcentagem de idiomas
df['language'].value_counts(normalize=True)


por             0.840859
não definido    0.139362
ger             0.007938
ita             0.004294
fre             0.003513
spa             0.002212
eng             0.001301
ara             0.000260
syr             0.000130
yid             0.000130
Name: language, dtype: float64

Podemos visualizar de forma mais clara esses dados com um gráfico de barras:

In [35]:
# criar gráfico de barras com os dados de idiomas dos periódicos
fig1 = px.bar(df_lang, x='Idioma', y='quantidade', color='Idioma', title='Idiomas dos periódicos')
#limitar texto do eixo x a 20 caracteres
fig1.update_xaxes(tickangle=45, tickfont=dict(size=10))
fig1.show()
# salvar gráfico
fig1.write_image('./charts/fig1.png')

### Período de publicação

Vamos analisar os dados referentes aos períodos de publicação dos periódicos digitalizados. Esses dados possuem algumas características que tornam sua análise mais complexa. 

Primeiro, percebemos que existe uma falta de padronização dos dados. Alguns registros possuem apenas o ano de início, outros possuem o ano de início e fim, e esses dados são escritos de formas variadas.

Portanto, primeiramente vou efetuar uma limpeza e padronização dos dados. Para isso, vou excluir caracteres especiais e buscar padronizar os dados para o formato 'yyyy-yyyy'.

Assim, acredito minimizar os erros de análise e facilitar a visualização dos dados.

In [36]:
#  limpar dados do período de publicação usando regex
# excluir [, ], (, ), :
df['period'] = df['period'].replace(to_replace=r'[\[\]\(\):\?]', value='', regex=True)
#substituir ' - ' por '-'
df['period'] = df['period'].replace(to_replace=r' - ', value='-', regex=True)
#substituir ' a ' por '-'
df['period'] = df['period'].replace(to_replace=r' a ', value='-', regex=True)

Para tornar a análise e visualização mais eficiente, vou criar novas colunas para o ano de início e fim dos periódicos. 

Caso o registro não possua o ano de fim, vou considerar o ano de início como ano de fim.

In [37]:
# criar coluna com o ano de início da publicação
# se iniciar com dígito, pegar os 4 primeiros caracteres, senão, pegar do 2º ao 5º caractere
df['start_year'] = df['period'].str.extract(r'(\d{4})', expand=False).fillna(df['period'].str[1:5])

In [38]:
# criar coluna com o ano de término da publicação a partir da coluna period
# pegar 4 últimos dígitos que aparecem na coluna period usando regex
df['end_year'] = df['period'].str.extract(r'(\d{4})$', expand=False)

In [39]:
# se 'end_year' for nulo, pegar o ano de início da publicação
df['end_year'] = df['end_year'].fillna(df['start_year'])

Uma última limpeza final, para excluir possíveis caracteres que não sejam dígitos nas colunas de ano de início e fim e inserior o número 0 para os registros que não possuem ano de início e/ou fim.

In [40]:
#converte as colunas para string
df['start_year'] = df['start_year'].astype(str)
df['end_year'] = df['end_year'].astype(str)

# Substitui todos os caracteres que não são dígitos por uma string vazia
df['start_year'] = df['start_year'].replace(to_replace=r'\D', value='', regex=True)
df['end_year'] = df['end_year'].replace(to_replace=r'\D', value='', regex=True)

# substitui os valores vazios por 0
df['start_year'] = df['start_year'].replace(to_replace=r'^\s*$', value='0', regex=True)
df['end_year'] = df['end_year'].replace(to_replace=r'^\s*$', value='0', regex=True)

# converte as colunas para int
df['start_year'] = df['start_year'].astype(int)
df['end_year'] = df['end_year'].astype(int)


Vamos conferir a quantidade de registros por ano de início e fim, lembrando que o número 0 representa os registros que não possuem ano de início e/ou fim:

Primeiro, criar um dataframe com a contagem dos registros por ano de início:

In [45]:
# contar início da publicação
df_bdate = df['start_year'].value_counts().rename_axis('Ano').reset_index(name='quantidade')
# organizar por ano
df_bdate = df_bdate.sort_values(by=['Ano'])
df_bdate

,Ano,quantidade
0,0,425
181,9,2
200,83,1
210,87,1
199,88,1
...,...,...
177,2012,3
183,2014,2
213,2017,1
198,2021,1


Vamos avaliar os dados iniciais do dataframe para encontrar possíveis erros:

In [47]:
df_bdate.head(10)

,Ano,quantidade
0,0,425
181,9,2
200,83,1
210,87,1
199,88,1
196,95,1
208,1521,1
206,1691,1
203,1741,1
201,1763,1


Percemos que 5 registros possuem registros que fogem do oadrão `YYYY` e, sendo estatisticamente insignificantes, serão excluídos do dataframe.

In [49]:
# excluir registros com ano entre 2 e 999
df_bdate = df_bdate[(df_bdate['Ano'] < 1) | (df_bdate['Ano'] > 999)]
df_bdate

,Ano,quantidade
0,0,425
208,1521,1
206,1691,1
203,1741,1
201,1763,1
...,...,...
177,2012,3
183,2014,2
213,2017,1
198,2021,1


Faremos o mesmo para final do dataframe:

In [50]:
df_bdate.tail(10)

,Ano,quantidade
162,2005,6
186,2006,2
195,2007,1
187,2010,2
209,2011,1
177,2012,3
183,2014,2
213,2017,1
198,2021,1
211,2022,1


Não há erros no fim. Podemos prosseguir para a visualização dos dados.

In [61]:
# excluir registros menor que 1000
df_bdate = df_bdate[df_bdate['Ano'] > 999]

# Criar scatter plot com os dados de datas de publicação com as datas de publicação no eixo Y
fig2 = px.scatter(df_bdate, x='Ano', y='quantidade', title='Datas de início de publicação')
fig2.show()
# salvar gráfico
fig2.write_image('./charts/fig2.png')


Realizaremos os mesmo procedimentos para a coluna do ano de fim:

In [52]:
# contar fim da publicação
df_edate = df['end_year'].value_counts().rename_axis('Ano').reset_index(name='quantidade')
# organizar por ano
df_edate = df_edate.sort_values(by=['Ano'])

,Ano,quantidade
0,0,425
186,9,2
202,83,1
215,87,1
218,88,1
...,...,...
193,2021,2
179,2022,3
196,2023,2
205,4949,1


In [55]:
df_edate.head(10)

,Ano,quantidade
0,0,425
210,1521,1
217,1694,1
216,1741,1
214,1767,1
198,1784,1
194,1812,2
184,1813,3
185,1814,2
200,1815,1


In [56]:
df_edate.tail(10)

,Ano,quantidade
211,2013,1
203,2014,1
204,2015,1
188,2016,2
181,2017,3
180,2018,3
206,2019,1
207,2020,1
193,2021,2
179,2022,3


In [54]:
# excluir registros com ano entre 2 e 999 e acima de 2023
df_edate = df_edate[(df_edate['Ano'] < 1) | (df_edate['Ano'] > 999)]
df_edate = df_edate[df_edate['Ano'] < 2023]
df_edate

,Ano,quantidade
0,0,425
210,1521,1
217,1694,1
216,1741,1
214,1767,1
...,...,...
180,2018,3
206,2019,1
207,2020,1
193,2021,2


Vamos avaliar quantos registros possuem o mesmo ano de início e fim:

In [57]:
# contar quando início e fim da publicação são iguais
df[df['start_year'] == df['end_year']].count()

title          5694
subtitle       2874
place          5626
period         5274
editor         4155
periodicity    4507
language       5694
start_year     5694
end_year       5694
dtype: int64

Em 5694 registros, o ano de início da publicação é o mesmo do ano de término, indicando que 7265 periódicos que possuem data registrada no *dataframe*, apenas 1569 possuem data de início e término diferentes. Isso corresponde a uma porcentagem de 21,6% dos periódicos.

Vamos criar uma nova coluna contando a quantidade de anos presentes no acervo de cada periódico:

In [58]:
# calcular a diferença entre início e fim da publicação
df['diff'] = df['end_year'] - df['start_year']


In [59]:
# listar periódicos com maiores diferenças entre início e fim da publicação
df.sort_values(by='diff', ascending=False).head(20)

,title,subtitle,place,period,editor,periodicity,language,start_year,end_year,diff
5880,O Povo,orgam do Partido Popular,"Pitangui, MG",19924,NaN,Semanal,não definido,1992,9924,7932
4303,O Abaete : jornal noticioso a servico do progr...,NaN,"Abaeté, MG",1948-4949,[s.n.],NaN,por,1948,4949,3001
2489,Diário de Pernambuco,NaN,"Recife, PE",1825-1984,Diário de Pernambuco,Diária,não definido,1825,1984,159
2286,Correio Official de Goyaz,NaN,"Goiás, GO",1837-1943,Typ. Provincial,Desconhecida,por,1837,1943,106
3659,Jornal do Commercio,NaN,"Manaus, AM",1904-2007,Empresa Jornal do Commercio,Diária,não definido,1904,2007,103
3473,Imprensa e Lei,NaN,Lisboa [Portugal],1854-1956,NaN,Indeterminada,por,1854,1956,102
1307,Almanak do Ministerio da Marinha,NaN,"Rio de Janeiro, RJ",1858-1960,NaN,Anual,por,1858,1960,102
6990,Revista do Clube de Engenharia,NaN,"Rio de Janeiro, RJ",1887-1989,O Clube,NaN,por,1887,1989,102
770,A Nova Era,NaN,"Aracaju, SE",1889-1990,[s.n.],Desconhecida,por,1889,1990,101
2597,Documentos,Acta dos festejos civicos com que o povo de So...,"Sobral, CE",1822-1922,NaN,NaN,não definido,1822,1922,100


:warning: Analisar com mais atenção os dados sobre acervos com maior longevidade de publicação.

### Periodicidade

:warning: ainda é preciso limpar os dados da coluna 'frequency'

In [ ]:
# contar periodicidade
df['periodicity'].value_counts()

In [ ]:
# contar porcentagem de periodicidade
df['periodicity'].value_counts(normalize=True)

### Locais de publicação

:warning: ainda é preciso limpar os dados da coluna 'place'

In [ ]:
# contar locais de publicação
df['place'].value_counts()

### Editoras

:warning: ainda é preciso limpar os dados da coluna 'editor'

In [ ]:
# contar editor
df['editor'].value_counts()